In [2]:
## Import the required libraries
import time
import os
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime

## Specify the path to the directory containing the ChromeDriver executable
chrome_driver_directory = "C:/Users/moreno/Downloads/chromedriver-win64" #insert your own path here #User moreno: 'moren' / ramon: 'arnol'

## Add the ChromeDriver directory to the PATH environment variable
os.environ["PATH"] += os.pathsep + chrome_driver_directory

Merging Various Datasets from the season 23/24

In [ ]:

import pandas as pd

# Define the base paths for your files
base_path = 'data/'  # Replace with the path to your CSV files

# Lists of file names to be merged
matches_info_files = [f'{base_path}/matches_info_2023_2024_{i}.csv' for i in range(1, 4)]
match_events_files = [f'{base_path}/match_events_2023_2024_{i}.csv' for i in range(1, 4)]
lineups_files = [f'{base_path}/lineups_2023_2024_{i}.csv' for i in range(1, 4)]

# Function to merge CSV files and save the merged file
def merge_csv_files(file_list, output_file):
    # Initialize an empty list to store DataFrames
    dataframes_list = []
    for file in file_list:
        # Read the current CSV file and append it to the list
        dataframes_list.append(pd.read_csv(file))
    # Concatenate all DataFrames in the list
    merged_dataframe = pd.concat(dataframes_list, ignore_index=True)
    # Save the merged DataFrame to a new CSV file
    merged_dataframe.to_csv(output_file, index=False)

# Merge and save the CSV files
merge_csv_files(matches_info_files, f'{base_path}/matches_info_2023_2024_merged.csv')
merge_csv_files(match_events_files, f'{base_path}/match_events_2023_2024_merged.csv')
merge_csv_files(lineups_files, f'{base_path}/lineups_2023_2024_merged.csv')

print("Merging completed.")

Scraping Player Information for Season 23/24

In [ ]:
# List of club numbers to iterate over
club_numbers = [26, 61, 452, 2790, 527, 434, 257, 260, 504, 322, 5499, 242]

# Initialize the Chrome driver
driver = webdriver.Chrome()

# Initialize an empty DataFrame to hold all the scraped data
players = pd.DataFrame()

for club_number in club_numbers:
    # Construct the URL with the current club number
    url = f'https://www.transfermarkt.com/servette-fc/kader/verein/{club_number}/saison_id/2023/plus/1'

    # Navigate to the page
    driver.get(url)

    # Wait for the page to load
    time.sleep(2) 

    # Try to find and interact with the iframe
    try:
        wait = WebDriverWait(driver, 2)  # Reduced wait time for iframe
        iframe = wait.until(EC.presence_of_element_located((By.ID, "sp_message_iframe_953358")))
        driver.switch_to.frame(iframe)

        # Wait for the 'Accept & continue' button to be clickable inside the iframe
        accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'accept')]")))
        accept_button.click()

        # Switch back to the main document
        driver.switch_to.default_content()
        print("Iframe found and handled.")
    except:
        print("No iframe found, continuing.")

    # Extract the club name using the specified XPath
    club_name_element = driver.find_element(By.XPATH, '//*[@id="main"]/main/header/div[1]/h1')
    club_name = club_name_element.text

    # Find the table by its XPath or CSS Selector
    table = driver.find_element(By.XPATH, '//*[@id="yw1"]/table')

    # Initialize a list to store all rows of the table
    table_data = []

    # Locate all rows of the table
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Loop through each row in the table
    for row in rows:
        # Get the text of each cell in the row
        row_data = [td.text for td in row.find_elements(By.TAG_NAME, "td")]
        table_data.append(row_data)

    # Convert the list of rows to a pandas DataFrame
    df = pd.DataFrame(table_data)

    # Drop the unnecessary columns (adjust indices as needed)
    df.drop(df.columns[[0, 1, 2, 6, 10]], axis=1, inplace=True)

    # Rename the remaining columns
    df.columns = ['Name', 'Position', 'Birthdate', 'Height', 'Foot', 'ContractBegin', 'ContractEnd', 'Value']

    # Drop rows where all the elements are 'None'
    df.dropna(how='all', inplace=True)

    # Add new columns with the club name and club number extracted from the page
    df['Club'] = club_name
    df['Club_Number'] = club_number

    # Append the DataFrame to the 'players' DataFrame
    players = pd.concat([players, df], ignore_index=True)

# Save the complete DataFrame to a CSV file after collecting all data
current_date = datetime.now().strftime('%Y-%m-%d')
filename = f'./data/players_{current_date}.csv'
players.to_csv(filename, index=False)

# Close the driver after scraping is done
driver.quit()

# Print a success message
print("Webscraping successfully completed")
print(players.head())


Cleaning Players Dataset for Season 23/24

In [9]:
# Function to clean and convert the market values
def clean_market_value(value_str):
    value_str = value_str.replace('€', '').replace('â‚¬', '')  # Remove the euro sign
    market_value = 0.0
    if 'm' in value_str:
        market_value = float(value_str.replace('m', ''))
    elif 'k' in value_str:
        market_value = float(value_str.replace('k', '')) / 1000
    return market_value



# Function to extract birthdate and age from the Birthdate column
def extract_birthdate_and_age(birthdate_str):
    birthdate_part, age_part = birthdate_str.split('(')
    birthdate = birthdate_part.strip()
    age = int(age_part.replace(')', '').strip())
    return birthdate, age

# Function to parse date that includes error handling
def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%b %d, %Y', errors='coerce')
    except ValueError:
        return pd.NaT

# Assuming the CSV file is in the same directory as the script, or provide the full path.
file_path = 'data/players_2024-04-06.csv'  # Replace with your file's actual path if different

# Load the data
df = pd.read_csv(file_path)

# Apply the function to clean and convert market values
df['Value'] = df['Value'].apply(clean_market_value)

# Apply the function to split the 'Birthdate' column into birthdate and age columns
df['Birthdate'], df['Age'] = zip(*df['Birthdate'].apply(extract_birthdate_and_age))

# Parse 'Birthdate', 'ContractBegin', and 'ContractEnd' columns as dates with error handling
df['Birthdate'] = df['Birthdate'].apply(parse_date)

# Add a new column 'season' with the value '2022_2023' for all rows
df['Season'] = '2023_2024'

# Save the modified DataFrame to a new CSV file
df.to_csv('data/players_2023_2024_cleaned.csv', index=False)

Cleaning merged data files and adding playing minutes   


In [10]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("./data/matches_info_2023_2024_merged.csv")

# Split the "Result" column into two new columns
df[['result_end', 'result_ht']] = df['Result'].str.split(expand=True)

# Remove the parentheses from the "result_ht" column
df['result_ht'] = df['result_ht'].str.strip('()')

# Extract goals for home team and away team
df[['goals_HomeTeam', 'goals_AwayTeam']] = df['result_end'].str.split(':', expand=True)

# Save the cleaned data to a new CSV file
df.to_csv("./data/matches_info_2023_2024_cleaned.csv", index=False)

# Display the first few rows of the cleaned data
print(df.head())

   Match ID                Home Team          Away Team      Result  \
0   4089693  Grasshopper Club Zurich        Servette FC  1:3\n(1:1)   
1   4089694       FC St. Gallen 1879      FC Basel 1893  2:1\n(1:1)   
2   4089695            FC Winterthur          FC Luzern  0:0\n(0:0)   
3   4089696           BSC Young Boys  FC Lausanne-Sport  2:1\n(0:0)   
4   4089697                FC Zürich   Yverdon Sport FC  2:0\n(1:0)   

         League result_end result_ht goals_HomeTeam goals_AwayTeam  
0  Super League        1:3       1:1              1              3  
1  Super League        2:1       1:1              2              1  
2  Super League        0:0       0:0              0              0  
3  Super League        2:1       0:0              2              1  
4  Super League        2:0       1:0              2              0  


In [11]:
# Read the CSV file
df_lineups = pd.read_csv("./data/lineups_2023_2024_merged.csv")

# Split the "Gameday" column to extract matchday, weekday, date, and time
df_lineups[['matchday', 'weekday', 'date', 'time']] = df_lineups['Gameday'].str.split('|').apply(lambda x: pd.Series([x[0].strip(), x[1].strip().split(',')[0].strip(), x[1].strip().split(',')[1].strip(), x[2].strip()]))

# Convert the "date" column to proper datetime format
df_lineups['date'] = pd.to_datetime(df_lineups['date'])

# Save the cleaned data to a new CSV file
df_lineups.to_csv("./data/lineups_2023_2024_cleaned.csv", index=False)

# Display the first few rows of the cleaned data
print(df_lineups.head())

      Position           Player  Age Market Value                     Club  \
0   Goalkeeper    Justin Hammel   22        €500k  Grasshopper Club Zurich   
1  Centre-Back  Kristers Tobers   22        €350k  Grasshopper Club Zurich   
2  Centre-Back       Ayumu Seko   23       €3.00m  Grasshopper Club Zurich   
3    Left-Back  Michael Kempter   28        €300k  Grasshopper Club Zurich   
4   Right-Back       Dirk Abels   26        €650k  Grasshopper Club Zurich   

                                   Gameday   H/A    Status  Match ID  \
0  1. Matchday | Sat, 7/22/23   |  6:00 PM  Home  Starting   4089693   
1  1. Matchday | Sat, 7/22/23   |  6:00 PM  Home  Starting   4089693   
2  1. Matchday | Sat, 7/22/23   |  6:00 PM  Home  Starting   4089693   
3  1. Matchday | Sat, 7/22/23   |  6:00 PM  Home  Starting   4089693   
4  1. Matchday | Sat, 7/22/23   |  6:00 PM  Home  Starting   4089693   

      matchday weekday       date     time  
0  1. Matchday     Sat 2023-07-22  6:00 PM  
1  1. Ma

C:\Users\moren\AppData\Local\Temp\ipykernel_39780\1852383721.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_lineups['date'] = pd.to_datetime(df_lineups['date'])


Merging Lineups Info with Matches Info to get Playing Minutes of each Player Season 23/24

In [12]:
# Read the CSV file
df_matches = pd.read_csv("./data/matches_info_2023_2024_cleaned.csv")
df_lineups = pd.read_csv("./data/lineups_2023_2024_cleaned.csv")

In [13]:
import pandas as pd

# Load the datasets
df_lineups = pd.read_csv('./data/lineups_2023_2024_cleaned.csv')
events_df = pd.read_csv('./data/match_events_2023_2024_merged.csv')

# Initialize 'Minutes Played' based on whether the player is in the starting lineup or not
df_lineups['Minutes Played'] = df_lineups['Status'].apply(lambda status: 90 if status == 'Starting' else 0)

# Filter the events DataFrame to only include substitution events
substitutions_df = events_df[events_df['Event'] == 'Substitution']

# Update 'Minutes Played' for players substituted out
for _, substitution in substitutions_df.iterrows():
    match_id = substitution['Match ID']
    player_out = substitution['Player Out']
    timestamp = int(substitution['Timestamp'].rstrip("'"))  # Remove the minute symbol and convert to int

    # Update minutes for the player substituted out
    df_lineups.loc[(df_lineups['Match ID'] == match_id) & (df_lineups['Player'] == player_out), 'Minutes Played'] = timestamp

# Update 'Minutes Played' for players substituted in
for _, substitution in substitutions_df.iterrows():
    match_id = substitution['Match ID']
    player_in = substitution['Player Event']
    timestamp = int(substitution['Timestamp'].rstrip("'"))  # Remove the minute symbol and convert to int

    # Update minutes for the player substituted in
    df_lineups.loc[(df_lineups['Match ID'] == match_id) & (df_lineups['Player'] == player_in), 'Minutes Played'] = 90 - timestamp

# Create a new 'Player Event' column based on the 'Status' and 'Minutes Played' columns
df_lineups['Player Event'] = ''
df_lineups.loc[(df_lineups['Status'] == 'Starting') & (df_lineups['Minutes Played'] != 90), 'Player Event'] = 'Substituted Out'
df_lineups.loc[(df_lineups['Status'] == 'Substitute') & (df_lineups['Minutes Played'] != 0), 'Player Event'] = 'Substituted In'

# Save the updated dataframe to a new CSV
df_lineups.to_csv('./data/lineups_2023_2024_SL_cleaned.csv', index=False)


Calculating Playing Minutes per Player Season 22/23

In [70]:
import pandas as pd
from collections import Counter

# Load the CSV file into a DataFrame
df = pd.read_csv('data/lineups_2022_2023_SL_cleaned.csv')

# Initialize the new columns with default values
df['Starting'] = df['Status'] == 'Starting'
df['Substitute'] = df['Status'] == 'Substitute'
df['Substituted In'] = df['Player Event'] == 'Substituted In'
df['Substituted Out'] = df['Player Event'] == 'Substituted Out'

# Convert boolean columns to integers
df['Starting'] = df['Starting'].astype(int)
df['Substitute'] = df['Substitute'].astype(int)
df['Substituted In'] = df['Substituted In'].astype(int)
df['Substituted Out'] = df['Substituted Out'].astype(int)

# Function to get the mode of the position
def get_mode(series):
    if len(series) == 0:
        return None
    return Counter(series).most_common(1)[0][0]

# Group by player and calculate the sum for all required columns
aggregations = {
    'Minutes Played': 'sum',
    'Starting': 'sum',
    'Substitute': 'sum',
    'Substituted In': 'sum',
    'Substituted Out': 'sum',
    'Position': lambda x: get_mode(x),  # Using custom function to get the mode
    'Age': 'max',  # Assuming Age does not change within the same season
    'Market Value': 'max',  # Assuming Market Value does not change within the same season
    'Club': 'max'  # Assuming players don't change clubs within the season
}

final_df = df.groupby('Player', as_index=False).agg(aggregations)

# Save the final dataset to a CSV file
final_df.to_csv('data/playing_minutes_2022_2023.csv', index=False)


Calculating Playing Minutes per Player Season 23/24

In [71]:
import pandas as pd
from collections import Counter

# Load the CSV file into a DataFrame
df = pd.read_csv('data/lineups_2023_2024_SL_cleaned.csv')

df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['Market Value'] = pd.to_numeric(df['Market Value'], errors='coerce')

# Initialize the new columns with default values
df['Starting'] = df['Status'] == 'Starting'
df['Substitute'] = df['Status'] == 'Substitute'
df['Substituted In'] = df['Player Event'] == 'Substituted In'
df['Substituted Out'] = df['Player Event'] == 'Substituted Out'

# Convert boolean columns to integers
df['Starting'] = df['Starting'].astype(int)
df['Substitute'] = df['Substitute'].astype(int)
df['Substituted In'] = df['Substituted In'].astype(int)
df['Substituted Out'] = df['Substituted Out'].astype(int)

# Function to get the mode of the position
def get_mode(series):
    if len(series) == 0:
        return None
    return Counter(series).most_common(1)[0][0]

# Group by player and calculate the sum for all required columns
aggregations = {
    'Minutes Played': 'sum',
    'Starting': 'sum',
    'Substitute': 'sum',
    'Substituted In': 'sum',
    'Substituted Out': 'sum',
    'Position': lambda x: get_mode(x),  # Using custom function to get the mode
    'Age': 'max',  # Assuming Age does not change within the same season
    'Market Value': 'max',  # Assuming Market Value does not change within the same season
    'Club': 'max'  # Assuming players don't change clubs within the season
}

final_df = df.groupby('Player', as_index=False).agg(aggregations)

# Save the final dataset to a CSV file
final_df.to_csv('data/playing_minutes_2023_2024.csv', index=False)


Merging Datasets for Player Minutes Overview 22/23 season

In [72]:
import pandas as pd

# Load the 'players' CSV file into a DataFrame
players_df = pd.read_csv('data/players_2022_2023_cleaned.csv')

# Load the 'playing minutes' CSV file into a DataFrame
playing_minutes_df = pd.read_csv('data/playing_minutes_2022_2023.csv')

# Rename 'Value' column to 'Market Value' in the playing minutes DataFrame to match the players DataFrame
playing_minutes_df.rename(columns={'Market Value': 'Value'}, inplace=True)

# Ensure there is no leading/trailing whitespace in the 'Player' columns
players_df['Name'] = players_df['Name'].str.strip()
playing_minutes_df['Player'] = playing_minutes_df['Player'].str.strip()

# Merge the DataFrames on the player's name
merged_df = pd.merge(playing_minutes_df, players_df, left_on='Player', right_on='Name', how='left')

# Columns that are in both DataFrames and need to be updated
columns_to_update = ['Position', 'Club', 'Value', 'Age']

# Update the values from players_df to the merged_df where they exist
for column in columns_to_update:
    merged_df[column + '_x'].update(merged_df[column + '_y'])

# Rename columns to remove the '_x' and '_y' suffixes from the merge
for column in merged_df.columns:
    if '_x' in column:
        merged_df.rename(columns={column: column.rstrip('_x')}, inplace=True)
    elif '_y' in column:
        merged_df.drop(columns=[column], inplace=True)

# Add 'Season' column with "2023_2024" for all entries
merged_df['Season'] = "2022_2023"

# Fill NaN values with 0 for playing minutes or other events recorded columns
columns_to_fill = ['Minutes Played', 'Starting', 'Substitute', 'Substituted In', 'Substituted Out']
for column in columns_to_fill:
    if column in merged_df.columns:
        merged_df[column].fillna(0, inplace=True)

# Convert float columns to int if needed, since we can't have fractions of counts
columns_to_convert = ['Minutes Played', 'Starting', 'Substitute', 'Substituted In', 'Substituted Out']
for column in columns_to_convert:
    if column in merged_df.columns:
        merged_df[column] = merged_df[column].astype(int)

columns_to_drop = ['Name']
merged_df.drop(columns_to_drop, axis=1, inplace=True)

# Save the resulting DataFrame to a new CSV file
merged_df.to_csv('data/players_2022_2023_for_meeting.csv', index=False)


C:\Users\moren\AppData\Local\Temp\ipykernel_39780\1617339444.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df[column + '_x'].update(merged_df[column + '_y'])
C:\Users\moren\AppData\Local\Temp\ipykernel_39780\1617339444.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a cop

Merging Datasets for Player Minutes Overview 23/24 season

In [73]:
import pandas as pd

# Load the 'players' CSV file into a DataFrame
players_df = pd.read_csv('data/players_2023_2024_cleaned.csv')

# Load the 'playing minutes' CSV file into a DataFrame
playing_minutes_df = pd.read_csv('data/playing_minutes_2023_2024.csv')

# Rename 'Value' column to 'Market Value' in the playing minutes DataFrame to match the players DataFrame
playing_minutes_df.rename(columns={'Market Value': 'Value'}, inplace=True)

# Ensure there is no leading/trailing whitespace in the 'Player' columns
players_df['Name'] = players_df['Name'].str.strip()
playing_minutes_df['Player'] = playing_minutes_df['Player'].str.strip()

# Merge the DataFrames on the player's name
merged_df = pd.merge(playing_minutes_df, players_df, left_on='Player', right_on='Name', how='left')

# Columns that are in both DataFrames and need to be updated
columns_to_update = ['Position', 'Club', 'Value', 'Age']

# Update the values from players_df to the merged_df where they exist
for column in columns_to_update:
    merged_df[column + '_x'].update(merged_df[column + '_y'])

# Rename columns to remove the '_x' and '_y' suffixes from the merge
for column in merged_df.columns:
    if '_x' in column:
        merged_df.rename(columns={column: column.rstrip('_x')}, inplace=True)
    elif '_y' in column:
        merged_df.drop(columns=[column], inplace=True)

# Add 'Season' column with "2023_2024" for all entries
merged_df['Season'] = "2023_2024"

# Fill NaN values with 0 for playing minutes or other events recorded columns
columns_to_fill = ['Minutes Played', 'Starting', 'Substitute', 'Substituted In', 'Substituted Out']
for column in columns_to_fill:
    if column in merged_df.columns:
        merged_df[column].fillna(0, inplace=True)

# Convert float columns to int if needed, since we can't have fractions of counts
columns_to_convert = ['Minutes Played', 'Starting', 'Substitute', 'Substituted In', 'Substituted Out']
for column in columns_to_convert:
    if column in merged_df.columns:
        merged_df[column] = merged_df[column].astype(int)

columns_to_drop = ['Name']
merged_df.drop(columns_to_drop, axis=1, inplace=True)

# Save the resulting DataFrame to a new CSV file
merged_df.to_csv('data/players_2023_2024_for_meeting.csv', index=False)


C:\Users\moren\AppData\Local\Temp\ipykernel_39780\3798695135.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df[column + '_x'].update(merged_df[column + '_y'])
C:\Users\moren\AppData\Local\Temp\ipykernel_39780\3798695135.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a cop